# ParkSense: Data Preparation & Cleaning

**Goal**: Prepare the historical parking sensor data for model training.

**Steps**:
1.  **Load Data**: Import the Static Bays dataset and the Historical Sensor dataset (2019).
2.  **Clean**: Handle missing values and ensure correct data types.
3.  **Merge**: Join datasets on `KerbsideID` to add geospatial data (Lat/Lon) to sensor readings.
4.  **Feature Engineering**: Extract time-based features (Hour, Day, Weekend).
5.  **Target Creation**: Create the prediction target (`is_free_15m`).
6.  **Export**: Save the processed dataset for the next stage (EDA & Training).

In [11]:
import pandas as pd
import numpy as np
import os

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## 1. Load Datasets
We use two main datasets:
*   **Static Bays**: Contains location info (`Latitude`, `Longitude`) for each `KerbsideID`.
*   **Historical Sensors**: Contains the event logs (Arrivals/Departures) for 2019.

In [12]:
# Paths
BAYS_PATH = '../data/on-street-parking-bays.csv'
SENSORS_PATH = '../data/On-street_Car_Parking_Sensor_Data_-_2019.csv' 

# Load Static Data
print("Loading Static Bays data...")
bays_df = pd.read_csv(BAYS_PATH)
print(f"Bays loaded: {bays_df.shape}")

# Load Historical Data (Random Sample)
# Note: The 2019 file is large. We load a random 1% sample for efficient development.
if os.path.exists(SENSORS_PATH):
    print("Loading Historical Sensor data (random sample)...")
    
    # Logic: Keep header (row 0) and randomly select 1% of other rows
    import random
    p = 0.01  # 1% of the lines
    
    # skiprows accepts a function. Returns True to skip the row.
    # We keep row 0 (header) and keep rows where random > p
    sensors_df = pd.read_csv(
        SENSORS_PATH, 
        parse_dates=['ArrivalTime', 'DepartureTime'], 
        skiprows=lambda i: i > 0 and random.random() > p
    )
    
    # RENAME COLUMNS to match our schema
    # The 2019 file uses 'ArrivalTime', 'DepartureTime', 'BayId'
    # We want 'Arrival_Time', 'Departure_Time', 'KerbsideID'
    sensors_df.rename(columns={
        'ArrivalTime': 'Arrival_Time', 
        'DepartureTime': 'Departure_Time',
        'BayId': 'KerbsideID'
    }, inplace=True)
    
    print(f"Sensors loaded (sample): {sensors_df.shape}")
    print(f"Columns: {list(sensors_df.columns)}")
else:
    print(f"WARNING: {SENSORS_PATH} not found. Please download the 2019 dataset.")
    # Create dummy data for demonstration if file is missing
    sensors_df = pd.DataFrame()

Loading Static Bays data...
Bays loaded: (23864, 7)
Loading Historical Sensor data (random sample)...


C:\Users\hsueh\AppData\Local\Temp\ipykernel_25620\4077230887.py:21: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  sensors_df = pd.read_csv(


Sensors loaded (sample): (426362, 20)
Columns: ['DeviceId', 'Arrival_Time', 'Departure_Time', 'DurationMinutes', 'StreetMarker', 'SignPlateID', 'Sign', 'AreaName', 'StreetId', 'StreetName', 'BetweenStreet1ID', 'BetweenStreet1', 'BetweenStreet2ID', 'BetweenStreet2', 'SideOfStreet', 'SideOfStreetCode', 'SideName', 'KerbsideID', 'InViolation', 'VehiclePresent']


In [13]:
print('--- Static Bays Data ---')
display(bays_df.head())

print('\n--- Historical Sensor Data ---')
display(sensors_df.head())

--- Static Bays Data ---


,RoadSegmentID,KerbsideID,RoadSegmentDescription,Latitude,Longitude,LastUpdated,Location
0,23322,NaN,Docklands Drive between Docklands Drive and We...,-37.816019,144.935552,2023-10-31,"-37.8160191, 144.9355523"
1,22124,NaN,Hartley Street between Lorimer Street and West...,-37.824862,144.940270,2023-10-31,"-37.8248623, 144.9402698"
2,22124,NaN,Hartley Street between Lorimer Street and West...,-37.824362,144.940408,2023-10-31,"-37.8243625, 144.9404083"
3,22124,NaN,Hartley Street between Lorimer Street and West...,-37.824469,144.940379,2023-10-31,"-37.8244685, 144.9403789"
4,22124,NaN,Hartley Street between Lorimer Street and West...,-37.824325,144.940188,2023-10-31,"-37.8243252, 144.9401876"



--- Historical Sensor Data ---


,DeviceId,Arrival_Time,Departure_Time,DurationMinutes,StreetMarker,SignPlateID,Sign,AreaName,StreetId,StreetName,BetweenStreet1ID,BetweenStreet1,BetweenStreet2ID,BetweenStreet2,SideOfStreet,SideOfStreetCode,SideName,KerbsideID,InViolation,VehiclePresent
0,23916,2019-02-23 04:18:47,2019-02-23 04:20:26,2,2334N,NaN,NaN,Windsor,123,BOURKE STREET,647,EXHIBITION STREET,1288.0,SPRING STREET,3,N,North,1728,False,True
1,23913,2019-12-13 17:54:01,2019-12-13 17:54:30,0,1581S,NaN,NaN,Banks,670,FLINDERS STREET,1171,QUEEN STREET,627.0,ELIZABETH STREET,4,S,South,1399,False,True
2,23915,2019-06-25 13:02:00,2019-06-25 13:02:20,0,2374N,287.0,1P MTR M-SAT 7:30-18:30,Windsor,123,BOURKE STREET,1221,RUSSELL STREET,647.0,EXHIBITION STREET,3,N,North,1753,False,True
3,23916,2019-02-07 18:08:00,2019-02-07 18:15:49,7,2334N,33.0,2P DIS M-SUN 7:30-19:30,Windsor,123,BOURKE STREET,647,EXHIBITION STREET,1288.0,SPRING STREET,3,N,North,1728,False,True
4,23915,2019-10-20 13:59:39,2019-10-20 15:00:54,61,2374N,3.0,1P SUN 7:30-18:30,Windsor,123,BOURKE STREET,1221,RUSSELL STREET,647.0,EXHIBITION STREET,3,N,North,1753,False,True


## 2. Data Cleaning
*   Ensure `KerbsideID` is present and consistent.
*   Convert timestamps to datetime objects.

In [14]:
# Drop rows without KerbsideID
sensors_df = sensors_df.dropna(subset=['KerbsideID'])
bays_df = bays_df.dropna(subset=['KerbsideID'])

# Ensure KerbsideID is string for merging (handling '7570N' etc.)
# We remove any trailing '.0' from float conversions if they exist
sensors_df['KerbsideID'] = sensors_df['KerbsideID'].astype(str).str.replace(r'\.0$', '', regex=True)
bays_df['KerbsideID'] = bays_df['KerbsideID'].astype(str).str.replace(r'\.0$', '', regex=True)

print("Cleaning complete.")

Cleaning complete.


## 3. Merge Datasets
Join the sensor events with the static bay data to get `Latitude` and `Longitude`.

In [15]:
# Select relevant columns from bays to avoid clutter
bays_cols = ['KerbsideID', 'Latitude', 'Longitude', 'RoadSegmentDescription']

# Merge
merged_df = sensors_df.merge(bays_df[bays_cols], on='KerbsideID', how='inner')

print(f"Merged shape: {merged_df.shape}")
merged_df.head()

Merged shape: (4032, 23)


,DeviceId,Arrival_Time,Departure_Time,DurationMinutes,StreetMarker,SignPlateID,Sign,AreaName,StreetId,StreetName,BetweenStreet1ID,BetweenStreet1,BetweenStreet2ID,BetweenStreet2,SideOfStreet,SideOfStreetCode,SideName,KerbsideID,InViolation,VehiclePresent,Latitude,Longitude,RoadSegmentDescription
0,24183,2019-09-28 21:12:35,2019-09-28 21:40:37,28,12813S,NaN,NaN,Drummond,1102,PELHAM STREET,900,LYGON STREET,610.0,DRUMMOND STREET,4,S,South,5926,False,False,-37.811834,144.975852,St Andrews Place between Lansdowne Street and ...
1,24183,2019-11-29 00:00:00,2019-11-29 07:30:00,450,12813S,NaN,NaN,Drummond,1102,PELHAM STREET,900,LYGON STREET,610.0,DRUMMOND STREET,4,S,South,5926,False,False,-37.811834,144.975852,St Andrews Place between Lansdowne Street and ...
2,24183,2019-10-03 14:57:57,2019-10-03 18:30:00,213,12813S,217.0,2P M-F 7:30-18:30,Drummond,1102,PELHAM STREET,900,LYGON STREET,610.0,DRUMMOND STREET,4,S,South,5926,False,False,-37.811834,144.975852,St Andrews Place between Lansdowne Street and ...
3,24183,2019-10-18 10:01:15,2019-10-18 11:26:18,85,12813S,217.0,2P M-F 7:30-18:30,Drummond,1102,PELHAM STREET,900,LYGON STREET,610.0,DRUMMOND STREET,4,S,South,5926,False,True,-37.811834,144.975852,St Andrews Place between Lansdowne Street and ...
4,24183,2019-08-23 13:03:24,2019-08-23 14:23:05,80,12813S,217.0,2P M-F 7:30-18:30,Drummond,1102,PELHAM STREET,900,LYGON STREET,610.0,DRUMMOND STREET,4,S,South,5926,False,True,-37.811834,144.975852,St Andrews Place between Lansdowne Street and ...


## 4. Feature Engineering
Extract useful features from the timestamp:
*   **Hour**: 0-23
*   **Day of Week**: 0 (Mon) - 6 (Sun)
*   **Is Weekend**: 1 if Sat/Sun, else 0

In [16]:
# Use Arrival_Time as the reference point for the 'state' of the system
merged_df['hour'] = merged_df['Arrival_Time'].dt.hour
merged_df['day_of_week'] = merged_df['Arrival_Time'].dt.dayofweek
merged_df['is_weekend'] = merged_df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

# Preview
merged_df[['Arrival_Time', 'hour', 'day_of_week', 'is_weekend']].head()

,Arrival_Time,hour,day_of_week,is_weekend
0,2019-09-28 21:12:35,21,5,1
1,2019-11-29 00:00:00,0,4,0
2,2019-10-03 14:57:57,14,3,0
3,2019-10-18 10:01:15,10,4,0
4,2019-08-23 13:03:24,13,4,0


## 5. Target Creation (The Tricky Part)
We want to predict: *"Will this bay be free in X minutes?"*

Since the 2019 data is event-based (Arrival/Departure), we need to transform it.
For a simple baseline, we can look at the **Duration**.

If a car arrives at `10:00` and stays for `60 mins` (departs `11:00`):
*   At `10:00` (Arrival), looking ahead 15 mins (`10:15`), the bay is **Occupied (0)**.
*   At `10:00`, looking ahead 45 mins (`10:45`), the bay is **Occupied (0)**.
*   At `10:00`, looking ahead 70 mins (`11:10`), the bay is **Free (1)**.

However, training on just "Arrival" rows biases the model (it only sees when cars *start* parking).
A better approach for the Master Table is to sample the state of every bay at regular intervals (e.g., every 15 mins).

**Simplified Approach for MVP**:
We will calculate the `DurationMinutes` and use it to label the row.

In [17]:
# Calculate Duration in minutes
merged_df['duration_min'] = (merged_df['Departure_Time'] - merged_df['Arrival_Time']).dt.total_seconds() / 60.0

# Define lookahead windows
LOOKAHEAD_15 = 15
LOOKAHEAD_30 = 30
LOOKAHEAD_45 = 45

# Create Target: Is the bay free after X minutes?
# If Duration < X, then the car has left by then -> Free (1)
# If Duration >= X, the car is still there -> Occupied (0)

merged_df['is_free_15m'] = (merged_df['duration_min'] < LOOKAHEAD_15).astype(int)
merged_df['is_free_30m'] = (merged_df['duration_min'] < LOOKAHEAD_30).astype(int)
merged_df['is_free_45m'] = (merged_df['duration_min'] < LOOKAHEAD_45).astype(int)

# Check class balance
print("Class Balance (15m):")
print(merged_df['is_free_15m'].value_counts(normalize=True))

Class Balance (15m):
is_free_15m
1    0.558532
0    0.441468
Name: proportion, dtype: float64


## 6. Export Processed Data
Save the clean, feature-rich dataset for the next notebook (EDA & Modeling).

In [ ]:
OUTPUT_PATH = '../data/processed_parking_data.csv'
merged_df.to_csv(OUTPUT_PATH, index=False)
print(f"Saved processed data to {OUTPUT_PATH}")

Saved processed data to ../data/processed_parking_data.csv


In [19]:
merged_df.head()

,DeviceId,Arrival_Time,Departure_Time,DurationMinutes,StreetMarker,SignPlateID,Sign,AreaName,StreetId,StreetName,BetweenStreet1ID,BetweenStreet1,BetweenStreet2ID,BetweenStreet2,SideOfStreet,SideOfStreetCode,SideName,KerbsideID,InViolation,VehiclePresent,Latitude,Longitude,RoadSegmentDescription,hour,day_of_week,is_weekend,duration_min,is_free_15m,is_free_30m,is_free_45m
0,24183,2019-09-28 21:12:35,2019-09-28 21:40:37,28,12813S,NaN,NaN,Drummond,1102,PELHAM STREET,900,LYGON STREET,610.0,DRUMMOND STREET,4,S,South,5926,False,False,-37.811834,144.975852,St Andrews Place between Lansdowne Street and ...,21,5,1,28.033333,0,1,1
1,24183,2019-11-29 00:00:00,2019-11-29 07:30:00,450,12813S,NaN,NaN,Drummond,1102,PELHAM STREET,900,LYGON STREET,610.0,DRUMMOND STREET,4,S,South,5926,False,False,-37.811834,144.975852,St Andrews Place between Lansdowne Street and ...,0,4,0,450.000000,0,0,0
2,24183,2019-10-03 14:57:57,2019-10-03 18:30:00,213,12813S,217.0,2P M-F 7:30-18:30,Drummond,1102,PELHAM STREET,900,LYGON STREET,610.0,DRUMMOND STREET,4,S,South,5926,False,False,-37.811834,144.975852,St Andrews Place between Lansdowne Street and ...,14,3,0,212.050000,0,0,0
3,24183,2019-10-18 10:01:15,2019-10-18 11:26:18,85,12813S,217.0,2P M-F 7:30-18:30,Drummond,1102,PELHAM STREET,900,LYGON STREET,610.0,DRUMMOND STREET,4,S,South,5926,False,True,-37.811834,144.975852,St Andrews Place between Lansdowne Street and ...,10,4,0,85.050000,0,0,0
4,24183,2019-08-23 13:03:24,2019-08-23 14:23:05,80,12813S,217.0,2P M-F 7:30-18:30,Drummond,1102,PELHAM STREET,900,LYGON STREET,610.0,DRUMMOND STREET,4,S,South,5926,False,True,-37.811834,144.975852,St Andrews Place between Lansdowne Street and ...,13,4,0,79.683333,0,0,0
